# THINGSvision
This is the PyTorch version, you can find a TensorFlow example [here](https://colab.research.google.com/github/ViCCo-Group/THINGSvision/blob/master/doc/tensorflow.ipynb)

## Settings

### Install thingsvision and dependencies

In [ ]:
!pip install --upgrade thingsvision

In [ ]:
!pip install ipywidgets

In [ ]:
import torch
import os
import torch.nn as nn
import thingsvision.vision as vision
import numpy as np

from thingsvision.model_class import Model
from google.colab import drive
from typing import Any

### Image and feature directories

Specify both `path/to/images` (input directory) and `path/to/features` (output directory) on your Google Drive. 
The image directory is expected to contain images that are saved similarly to `/dog/img_1.png` or `/cat/img_1.jpg`. 

In [ ]:
image_dir = 'path/to/images'  # path/to/images in GDrive
output_dir = 'path/to/features' # path/to/output  in GDrive

Mount Google Drive 

In [ ]:
mounted_dir = '/thingsvision'
drive.mount(mounted_dir, force_remount=True)

In [ ]:
full_image_path = os.path.join(mounted_dir, 'MyDrive', image_dir)
full_output_path = os.path.join(mounted_dir, 'MyDrive', output_dir)

### Helper functions to extract features

In [ ]:
def extract_features(
                    model: Any,
                    module_name: str,
                    image_path: str,
                    out_path: str,
                    batch_size: int,
                    flatten_activations: bool,
                    apply_center_crop: bool,
                    clip: bool=False,
) -> np.ndarray:
    """Extract features for a single layer."""
    dl = vision.load_dl(
                        root=image_path,
                        out_path=out_path,
                        batch_size=batch_size,
                        transforms=model.get_transformations(apply_center_crop=apply_center_crop),
                        backend=backend,
    )
    # exctract features
    features, _ = model.extract_features(
                                        data_loader=dl,
                                        module_name=module_name,
                                        batch_size=batch_size,
                                        flatten_acts=flatten_activations,
                                        clip=clip,
                                        return_probabilities=False,
    )
    return features


def extract_all_layers(
                        model_name: str,
                        model: Any,
                        image_path: str,
                        out_path: str,
                        batch_size: int,
                        flatten_activations: bool,
                        apply_center_crop: bool,
                        layer: Any=nn.Linear,
                        clip: bool=False,
) -> None:
    """Extract features for all selected layers and save them to disk."""
    for module_name, module in model.model.named_modules():
        if isinstance(module, layer):
            # extract features for layer "module_name"
            features = extract_features(
                                        model=model,
                                        module_name=module_name,
                                        image_path=image_path,
                                        out_path=out_path,
                                        batch_size=batch_size,
                                        flatten_activations=flatten_activations,
                                        apply_center_crop=apply_center_crop,
                                        clip=clip,
            )
            # save features to disk
            vision.save_features(features, f'{out_path}/features_{model_name}_{module_name}', 'npy') 

### Variables

In [ ]:
backend = 'pt' # backend 'pt' for PyTorch or 'tf' for Tensorflow 
pretrained = True # use pretrained model weights
model_path = None # if pretrained = False (i.e., randomly initialized weights) set path to model weights
batch_size = 32 # use a power of two (this can be any size, depending on the number of images for which you aim to extract features)
apply_center_crop = True # center crop images (set to False, if you don't want to center-crop images)
flatten_activations = True # whether or not features (e.g., of Conv layers) should be flattened
device = 'cuda' if torch.cuda.is_available() else 'cpu'

Select `model` and `layer` for which you want to extract image features. If you want to extract features from a `torchvision` model, use the model naming defined [here](https://pytorch.org/vision/stable/models.html) (e.g., `vgg16` if you want to use VGG-16). If you are uncertain about the naming and enumeration of the layers, use `model.show()` to see how specific layers called.

### Example 1: VGG-16 with batch norm (pretrained on ImageNet)

In [ ]:
## load model
model_name = 'vgg16_bn' 
model = Model(
            model_name,
            pretrained=pretrained,
            model_path=model_path,
            device=device,
            backend=backend,
)

In [ ]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming
# module_name = model.show() 
module_name = 'features.23' 

#### Feature extraction single layer

In [ ]:
# extract features for a single layer
features = extract_features(
                            model=model,
                            module_name=module_name,
                            image_path=full_image_path,
                            out_path=full_output_path,
                            batch_size=batch_size,
                            flatten_activations=flatten_activations,
                            apply_center_crop=apply_center_crop,
                            clip=False,
)

# save features to disk
vision.save_features(features, f'{full_output_path}/features_{model_name}_{module_name}', 'npy')

#### Feature extraction all convolutional or fully-connected layers

In [ ]:
# extract features for all convolutional layers (i.e., Conv2d) and save them to disk
layer = nn.Conv2d
extract_all_layers(
                    model_name=model_name,
                    model=model,
                    image_path=full_image_path,
                    out_path=full_output_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    layer=layer,
                    clip=False,
)

In [ ]:
# extract features for all fully-connected layers (i.e., Linear) and save them to disk
layer = nn.Linear
extract_all_layers(
                    model_name=model_name,
                    model=model,
                    image_path=full_image_path,
                    out_path=full_output_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    layer=layer,
                    clip=False,
)

### Example 2: VGG-16 with batch norm (pretrained on Ecoset)

In [ ]:
## load model
model_name = 'VGG16bn_ecoset'
model = Model(
            model_name,
            pretrained=pretrained,
            model_path=model_path,
            device=device,
            backend=backend,
)

In [ ]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming
# module_name = model.show() 
module_name = 'features.23' 

#### Feature extraction single layer

In [ ]:
# extract features
features = extract_features(
                            model=model,
                            module_name=module_name,
                            image_path=full_image_path,
                            out_path=full_output_path,
                            batch_size=batch_size,
                            flatten_activations=flatten_activations,
                            apply_center_crop=apply_center_crop,
                            clip=False,
)

# save features to disk
vision.save_features(features, f'{full_output_path}/features_{model_name}_{module_name}', 'npy')

#### Feature extraction all convolutional or fully-connected layers

In [ ]:
# extract features for all convolutional layers (i.e., Conv2d) and save them to disk
layer = nn.Conv2d
extract_all_layers(
                    model_name=model_name,
                    model=model,
                    image_path=full_image_path,
                    out_path=full_output_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    layer=layer,
                    clip=False,
)

In [ ]:
# extract features for all fully-connected layers (i.e., Linear) and save them to disk
layer = nn.Linear
extract_all_layers(
                    model_name=model_name,
                    model=model,
                    image_path=full_image_path,
                    out_path=full_output_path,
                    batch_size=batch_size,
                    flatten_activations=flatten_activations,
                    apply_center_crop=apply_center_crop,
                    layer=layer,
                    clip=False,
)

### Example 3: CLIP (multimodal pretraining)

In [ ]:
# load model
model_name = 'clip-ViT'
model = Model(
            model_name,
            pretrained=pretrained,
            model_path=model_path,
            device=device,
            backend=backend,
)

In [ ]:
## select layer

# NOTE: uncomment the line below, if you are uncertain about layer naming
# module_name = model.show()
module_name = 'visual' # penultimate layer in CLIP

#### Feature extraction single layer

In [ ]:
# extract features
features = extract_features(
                            model=model,
                            module_name=module_name,
                            image_path=full_image_path,
                            out_path=full_output_path,
                            batch_size=batch_size,
                            flatten_activations=flatten_activations,
                            apply_center_crop=apply_center_crop,
                            clip=True,
)

# apply centering (not necessary, but may be desirable, depending on the analysis)
features = vision.center_features(features)

# save features to disk
vision.save_features(features, f'{full_output_path}/features_{model_name}_{module_name}', 'npy')

### Representational Similarity Analysis (RSA)

In [ ]:
# compute representational dissimilarity matrix
rdm = vision.compute_rdm(features, method='correlation')

In [ ]:
# plot rdm
vision.plot_rdm(
                out_path,
                features,
                method='correlation',
                format='.png', # '.jpg'
                colormap='cividis',
                show_plot=True,
)